## Assignment for Module 5, Training Models

In this assignment you will train different models on a given data set, and find the one that performs best

### Getting the data for the assignment (similar to the notebook from chapter 2 of Hands-On...)

In [214]:
import os
import tarfile
from six.moves import urllib
import numpy as np

# np.random.seed = 42

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [215]:
fetch_housing_data()

In [216]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [217]:
housing = load_housing_data()


## HW Q.1:

In [218]:
housing.shape

(20640, 10)

### Fix the categories in the categorical variable

In [219]:
d = {'<1H OCEAN':'LESS_1H_OCEAN', 'INLAND':'INLAND', 'ISLAND':'ISLAND', 'NEAR BAY':'NEAR_BAY', 'NEAR OCEAN':'NEAR_OCEAN'}
housing['ocean_proximity'] = housing['ocean_proximity'].map(lambda s: d[s])

### Add 2 more features

In [220]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["population_per_household"]=housing["population"]/housing["households"]

### Fix missing data

In [221]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True) 

/tmp/ipykernel_593246/3788917901.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing["total_bedrooms"].fillna(median, inplace=True)


### Create dummy variables based on the categorical variable

In [222]:
one_hot = pd.get_dummies(housing['ocean_proximity'])
housing = housing.drop('ocean_proximity', axis=1)
housing = housing.join(one_hot)

In [223]:
one_hot

,INLAND,ISLAND,LESS_1H_OCEAN,NEAR_BAY,NEAR_OCEAN
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,False,True,False
3,False,False,False,True,False
4,False,False,False,True,False
...,...,...,...,...,...
20635,True,False,False,False,False
20636,True,False,False,False,False
20637,True,False,False,False,False
20638,True,False,False,False,False


### Check the data

In [224]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   longitude                 20640 non-null  float64
 1   latitude                  20640 non-null  float64
 2   housing_median_age        20640 non-null  float64
 3   total_rooms               20640 non-null  float64
 4   total_bedrooms            20640 non-null  float64
 5   population                20640 non-null  float64
 6   households                20640 non-null  float64
 7   median_income             20640 non-null  float64
 8   median_house_value        20640 non-null  float64
 9   rooms_per_household       20640 non-null  float64
 10  population_per_household  20640 non-null  float64
 11  INLAND                    20640 non-null  bool   
 12  ISLAND                    20640 non-null  bool   
 13  LESS_1H_OCEAN             20640 non-null  bool   
 14  NEAR_B

# ASSIGNMENT

### 1. Partition into train and test

Use train_test_split from sklearn.model_selection to partition the dataset into 70% for training and 30% for testing.

You can use the 70% for training set as both training and validation by using cross-validation.


## HW Q.2:
train_set, test_set = train_test_split(housing, test_size=0.3, random_state=42)

In [250]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.3, random_state=42) ## YOUR CODE HERE ##

### Features

In [251]:
target = 'median_house_value'
features = list(train_set.columns)
features = [f for f in features if f!=target]

In [252]:
X_tr = train_set[features]
y_tr = train_set[[target]]

X_te = test_set[features]
y_te = test_set[[target]]

### 2. Polynomial transformations

Use PolynomialFeatures from sklearn.preprocessing

In [253]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
poly.fit(X_tr)
X_tr_poly = poly.transform(X_tr)  ## YOUR CODE HERE ##
X_te_poly = poly.transform(X_te)  ## YOUR CODE HERE ##

##### You should obtain X_tr and X_te with 136 columns each, since originally you had 15 features.

##### With m original features, the new added polynomial features of degree 2 are: $(m^2-m)/2+m+1$. Why?

##### These, plus the original features gives a total of  $(m^2-m)/2+2m+1$

In [254]:
print("Original number of features: "+str(len(features)))
print("Final number of features: "+str(X_tr_poly.shape[1]))

Original number of features: 15
Final number of features: 136


## HW Q.3:

In [256]:
X_tr_poly.shape

(14448, 136)

### 3. Scaling features

Similarly, use StandardScaler from sklearn.preprocessing to normalize the training and testing data, using the training data

In [257]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_tr_poly)
X_tr_poly_scaled = scaler.transform(X_tr_poly)
X_te_poly_scaled = scaler.transform(X_te_poly)

# scale original sets as well:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

#### Comparing models

In [258]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())

### 4. Linear regression on original features (no transformations) --- benchmark

#### Your goal is to find the model that minimizes the rmse score

In [259]:
from sklearn.linear_model import LinearRegression
lin_scores = cross_val_score(LinearRegression(), train_set[features], train_set[target], scoring="neg_mean_squared_error", cv=4)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [70142.55721218 67456.39127204 67318.3258893  70866.26065275]
Mean: 68945.8837565685


### 5. Linear regression  (on transformed features: polynomial transformation + scaling)

Now do as in 4 but with the original and transformed features (136 features)

#### HW Q.4:
Scores: [1.97664966e+16 8.66038819e+14 3.26684960e+15 5.08453658e+14]
Mean: 6101959672738182.

In [281]:
# lin_scores_poly_scaled = cross_val_score(LinearRegression(), X_tr_poly_scaled, y_tr, scoring="neg_mean_squared_error", cv=4)
lin_scores_poly_scaled = cross_val_score(LinearRegression(), X_tr_poly_scaled, y_tr, scoring="neg_mean_squared_error", cv=4)
lin_rmse_scores_poly_scaled = np.sqrt(-lin_scores_poly_scaled)
display_scores(lin_rmse_scores_poly_scaled) 

Scores: [1.97664966e+16 8.66038819e+14 3.26684960e+15 5.08453658e+14]
Mean: 6101959672738182.0


If the error on the cross-validation is too high it is because the model is over-fitting. Regularization is needed.

### 6. Ridge regression

In [241]:
from sklearn.linear_model import Ridge
param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000]}]
grid_search_rr = GridSearchCV(Ridge(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search_rr.fit(X_tr_poly_scaled, y_tr)

GridSearchCV(cv=3, estimator=Ridge(random_state=42),
             param_grid=[{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}],
             scoring='neg_mean_squared_error')

In [275]:
print(grid_search_rr.best_params_)
print(np.sqrt(-grid_search_rr.best_score_))

{'alpha': 1000}
67204.15300429483


In [244]:
param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000]}]
grid_search_rr = GridSearchCV(Ridge(), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search_rr.fit(X_tr_poly_scaled, y_tr)

print(grid_search_rr.best_params_)
print(np.sqrt(-grid_search_rr.best_score_))

{'alpha': 1000}
67204.15300429483


In [ ]:
# from sklearn.linear_model import Ridge
# param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000]}]
# grid_search_rr = GridSearchCV(Ridge(), param_grid, cv=3, scoring='neg_mean_squared_error')
# grid_search_rr.fit(X_tr_poly, y_tr)

# print(grid_search_rr.best_params_)
# print(np.sqrt(-grid_search_rr.best_score_))

### 7. Lasso regression

Now do the same as in 6 but with Lasso

In [276]:
from sklearn.linear_model import Lasso
# param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000]}]
param_grid = [{'alpha': [100,1000]}]
# grid_search_lr = GridSearchCV(Lasso(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search_lr = GridSearchCV(Lasso(), param_grid, cv=3, scoring='neg_mean_squared_error', verbose=True)

grid_search_lr.fit(X_tr_poly_scaled, y_tr)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/akochel/.local/share/virtualenvs/3253-Machine-Learning-4oSlvO5A/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.501e+11, tolerance: 1.292e+10
  model = cd_fast.enet_coordinate_descent(
/home/akochel/.local/share/virtualenvs/3253-Machine-Learning-4oSlvO5A/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.201e+12, tolerance: 1.290e+10
  model = cd_fast.enet_coordinate_descent(
/home/akochel/.local/share/virtualenvs/3253-Machine-Learning-4oSlvO5A/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did no

GridSearchCV(cv=3, estimator=Lasso(), param_grid=[{'alpha': [100, 1000]}],
             scoring='neg_mean_squared_error', verbose=True)

#### HW Q.5:

In [277]:
print(grid_search_lr.best_params_)
print(np.sqrt(-grid_search_lr.best_score_))
# grid_search_lr.

{'alpha': 1000}
66619.6312718291


#### HW Q.6:
4- Option 1, 2, and 3 are correct. 

### 8. Elastic Net regression

Do the same as in 6 and 7, but now with Elastic Net. However, the grid search should be over the parameters alpha and  l 1ratio. Use just 3 values for l1_ratio.

In [265]:
from sklearn.linear_model import ElasticNet
param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000], 'l1_ratio':[0.2,0.4,0.6]}]

grid_search_en = GridSearchCV(ElasticNet(), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search_en.fit(X_tr_poly_scaled, y_tr)

/home/akochel/.local/share/virtualenvs/3253-Machine-Learning-4oSlvO5A/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.236e+13, tolerance: 1.936e+10
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=3, estimator=ElasticNet(),
             param_grid=[{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
                          'l1_ratio': [0.2, 0.4, 0.6]}],
             scoring='neg_mean_squared_error')

#### HW Q.7:

In [282]:
print(grid_search_en.best_params_)
print(np.sqrt(-grid_search_en.best_score_))

{'alpha': 0.1, 'l1_ratio': 0.4}
67072.67698457319


### Evaluating your best model on TESTING data

Choose among grid_search_rr, grid_search_lr, and grid_search_enr, the model with best performance

In [ ]:
from sklearn.metrics import mean_squared_error

final_model = grid_search.best_estimator_   ## grid_search SHOULD BE THE BEST GRID SEARCH ##

y_te_estimation = final_model.predict(X_te)

final_mse = mean_squared_error(y_te, y_te_estimation)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x=y_te, y=y_te_estimation)
plt.xlim([-200000,800000])
plt.ylim([-200000,800000])
plt.show()

### Question: Before you computed the final_rmse on the test data, what was your expected value for this quantity? Does your best model have high variance?

##### YOUR ANSWER HERE 

#[Optional]
Why does the matrix X appears transponsed in the normal equation in the linear regression? Equation 4.4. Start from equation 4.3



#[Optional]
Do all Gradient Descent algorithms lead to the same model provided you let them run long enough?



#[Optional]
Is it a good idea to stop Mini-batch Gradient Descent immediately when the validation error goes up?



#[Optional]
Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter α or reduce it?



#[Optional]
Why does the matrix X appears transponsed in the normal equation in the linear regression? Equation 4.4. Start from equation 4.3

